In [1]:
import boto3
import numpy as np
import pandas as pd

#### Helper methods to filter and transform dataframes

In [2]:
# df should be a pandas dataframe
# cols_to_keep should be a list of column strings present in df
# assumes the input data frame will have a Status, Fall Term, and County columns
# returns all rows in the dataframe where Status = Admitted and Fall Term = 2018
# filters cols based on the cols_to_keep
# removes duplicates if remove_dups is True (by default)
def filter_dataframe(df, cols_to_keep, year=2016, remove_dups=True):
    # take students who were admitted or applied
    df = df[(df['Status'] == 'Adm') | (df['Status'] == 'App')]
    # only look at the most recent admits
    df = df[df['Fall Term'] ==  year]
    # filter columns
    df = df[cols_to_keep]
    
    if remove_dups:
        # remove duplicates
        df = df.drop_duplicates()
    
    print "Number of rows in the set {0}".format(df.size)
    print "Number of unique counties {0}".format(len(list(df['County'].unique())))
    
    return df

In [3]:
# transform gpas into discrete buckets
# round down to the lower bucket for each range
# buckets are low (1), medium (2), high (3)
# min GPA = 3.24, max GPA = 4.35, stddev = 0.2134
def bucket_gpa(gpa):
    if gpa >= 3.0 and gpa < 3.2:
        return 1

    if gpa >= 3.2 and gpa < 3.4:
        return 1

    if gpa >= 3.4 and gpa < 3.6:
        return 2

    if gpa >= 3.6 and gpa < 3.8:
        return 2

    if gpa >= 3.8 and gpa < 4.0:
        return 2

    if gpa >= 4.0 and gpa < 4.2:
        return 3

    return 3

In [4]:
# transform sat scores into discrete buckets
# low (1), med (2), high (3)
# min score = 440 and max score = 550
def bucket_sat_score(score):
    if score >= 400 and score < 440:
        return 1

    if score >= 440 and score < 460:
        return 1
    
    if score >= 460 and score < 480:
        return 2
    
    if score >= 480 and score < 500:
        return 2
    
    if score >= 500 and score < 520:
        return 2
    
    if score >= 520 and score < 540:
        return 3
    
    return 3

#### Ingest and Format Admit Ethnicity Data
#### Resulting DataFrame should have County and Ethnicity of all 2018 Admitted UC students

In [5]:
# import data by ethnicity
ethnicities = pd.read_csv('../data/HS_ethnicity_by_year_data.csv')
# rename County, Status, and Ethnicity Columns
ethnicities = ethnicities.rename(columns={'County/State/ Territory': 'County', 'Count': 'Status', 'Uad Uc Ethn 6 Cat': 'Ethnicity', 'Pivot Field Values': 'Count'})
# further filter rows
# only keep rows that have a county
ethnicities = ethnicities[pd.notnull(ethnicities['County'])]
ethnicities = filter_dataframe(ethnicities, ['County', 'Ethnicity', 'Status'])

Number of rows in the set 852
Number of unique counties 42


In [6]:
ethnicities.head()

,County,Ethnicity,Status
80,TX,White,Adm
81,TX,Asian,Adm
82,TX,Asian,App
83,TX,All,Adm
84,TX,All,App


In [7]:
# import data by gender
genders = pd.read_csv('../data/FR_GENDER_data.csv')
genders = genders.rename(columns={'Count': 'Status', 'Pivot Field Values': 'Count', 'County/State/ Territory': 'County'})
genders['Fall Term'] = 2016
genders = filter_dataframe(genders, ['County', 'Gender', 'Status'])

Number of rows in the set 999
Number of unique counties 55


In [8]:
genders.head()

,County,Gender,Status
1,Los Angeles,Male,Adm
2,Los Angeles,Male,App
4,Los Angeles,Female,Adm
5,Los Angeles,Female,App
7,Los Angeles,All,Adm


#### Min GPA for 2018 admits is 3.24
#### Max GPA for 2018 admits is 4.35

In [9]:
# import data by gpa
gpas = pd.read_csv('../data/FR_GPA_by_Inst_data.csv')
# rename columns
gpas = gpas.rename(columns={'Measure Values': 'GPA', 'School Name': 'School', 'Measure Names': 'Status'})
# filter any rows that do not have a GPA or county
gpas = gpas[pd.notnull(gpas['GPA'])]
gpas = gpas[gpas['County'] != 'Not Applicable']
# transform status to match the other datasets
gpas['Status'] = [status.replace('GPA', '').strip() for status in gpas['Status']]
gpas = filter_dataframe(gpas, ['County', 'GPA', 'Campus', 'Status'], remove_dups=False)
# take the average admitted gpa per county and campus
gpas = gpas.groupby(['County', 'Campus', 'Status']).mean().reset_index()
# round all the GPAs into discrete buckets
gpas['GPA'] = [bucket_gpa(gpa) for gpa in gpas['GPA']]
gpas = gpas.drop_duplicates()
print "Number of rows in the set {0}".format(gpas.size)

Number of rows in the set 7596
Number of unique counties 28
Number of rows in the set 1800


In [10]:
gpas.head()

,County,Campus,Status,GPA
0,Alameda,Berkeley,Adm,3
1,Alameda,Berkeley,App,2
2,Alameda,Davis,Adm,3
3,Alameda,Davis,App,2
4,Alameda,Irvine,Adm,3


#### SAT Score Source data
#### https://data.world/education/california-sat-report-2015-2016/workspace/project-summary?agentid=education&datasetid=california-sat-report-2015-2016

In [11]:
sat_scores = pd.read_csv('../data/sat-report-2015-2016.csv')
# rtype = Record Type: C=County, D=District, S=School, X=State
sat_scores = sat_scores[sat_scores['rtype'] == 'C']
# columns to keep
sat_scores = sat_scores[['AvgScrRead', 'AvgScrMath', 'AvgScrWrit', 'cname']]
# rename for join
sat_scores = sat_scores.rename(columns={'cname': 'County'})
# drop any rows that do not have a school
sat_scores = sat_scores[pd.notnull(sat_scores['County'])]
# convert score cols to ints
sat_scores['AvgScrRead'] = pd.to_numeric(sat_scores['AvgScrRead'], errors=coerce)
sat_scores['AvgScrMath'] = pd.to_numeric(sat_scores['AvgScrMath'], errors=coerce)
sat_scores['AvgScrWrit'] = pd.to_numeric(sat_scores['AvgScrWrit'], errors=coerce)
# drop null cols
sat_scores = sat_scores[pd.notnull(sat_scores['AvgScrRead'])]
sat_scores = sat_scores[pd.notnull(sat_scores['AvgScrMath'])]
sat_scores = sat_scores[pd.notnull(sat_scores['AvgScrWrit'])]
# bucket scores
sat_scores['AvgScrRead'] = [bucket_sat_score(score) for score in sat_scores['AvgScrRead']]
sat_scores['AvgScrMath'] = [bucket_sat_score(score) for score in sat_scores['AvgScrMath']]
sat_scores['AvgScrWrit'] = [bucket_sat_score(score) for score in sat_scores['AvgScrWrit']]
# remove duplicates
sat_scores = sat_scores.drop_duplicates()
sat_scores.head()

,AvgScrRead,AvgScrMath,AvgScrWrit,County
1,2,3,2,Alameda
89,3,3,2,Amador
95,2,3,2,Butte
119,2,2,2,Calaveras
126,1,2,1,Colusa


In [12]:
print "Number of rows in the set {0}".format(sat_scores.size)
print "Number of unique counties {0}".format(len(list(sat_scores['County'].unique())))

Number of rows in the set 224
Number of unique counties 56


In [13]:
# join the datasets
result = pd.merge(ethnicities, genders, on=['County', 'Status'])
result = pd.merge(result, gpas, on=['County', 'Status']).drop_duplicates()

# SAT scores do not have admittance data, so assume all applicants from
# the same county have the same SAT scores
result = pd.merge(result, sat_scores, on=['County']).drop_duplicates()

# convert admitted and applied to labels
# 0 for applied, 1 for admitted
result['Status'] = [0 if status == 'App' else 1 for status in result['Status']]
result.drop_duplicates(inplace=True)

In [14]:
result.head()

,County,Ethnicity,Status,Gender,Campus,GPA,AvgScrRead,AvgScrMath,AvgScrWrit
0,San Bernardino,Domestic Unknown,0,Male,Berkeley,2,2,2,1
1,San Bernardino,Domestic Unknown,0,Male,Davis,2,2,2,1
2,San Bernardino,Domestic Unknown,0,Male,Irvine,2,2,2,1
3,San Bernardino,Domestic Unknown,0,Male,Los Angeles,2,2,2,1
4,San Bernardino,Domestic Unknown,0,Male,Merced,1,2,2,1


In [15]:
result.size

53460

In [16]:
print ("Total number of admitted students {0}".format(len(result[result['Status'] == 1])))

Total number of admitted students 2855


In [17]:
print ("Total number of applied students {0}".format(len(result[result['Status'] == 0])))

Total number of applied students 3085


In [18]:
result.describe()

,Status,GPA,AvgScrRead,AvgScrMath,AvgScrWrit
count,5940.000000,5940.000000,5940.000000,5940.000000,5940.000000
mean,0.480640,2.318350,2.125926,2.363300,1.934680
std,0.499667,0.484305,0.382697,0.517418,0.544787
min,0.000000,1.000000,1.000000,1.000000,1.000000
25%,0.000000,2.000000,2.000000,2.000000,2.000000
50%,0.000000,2.000000,2.000000,2.000000,2.000000
75%,1.000000,3.000000,2.000000,3.000000,2.000000
max,1.000000,3.000000,3.000000,3.000000,3.000000


#### Resulting dataset has 2,485 rows
#### With county, ethnicity, gender, UC campus, GPA, SAT scores, and calculated admittance rate

In [19]:
result.to_csv('../data/applicant_admissions_data.csv')

In [20]:
S3 = boto3.client('s3', region_name='eu-central-1')
S3.upload_file('../data/applicant_admissions_data.csv', 'gosat-data', 'applicant_admissions_data.csv')

In [21]:
result['Campus'].describe()

count               5940
unique                10
top       Universitywide
freq                 648
Name: Campus, dtype: object